In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import os
from math import ceil
from rich.progress import track

# Disable chained assignment warning
pd.options.mode.chained_assignment = None

In [2]:
um_per_pixel = 0.117
s_per_frame = 2
window_size = 5
dtype_dict = {
    "t": "float64",
    "x": "float64",
    "y": "float64",
    "trackID": "Int64",
}

In [3]:
# File selection dialog (make sure to run this in an environment that supports dialogs, or replace with a direct file path)
csv_file_path = filedialog.askopenfilename(
    title="Select CSV File",
    filetypes=(("CSV files", "*.csv"), ("All files", "*.*")),
)

In [4]:
# Read the CSV file with predefined data types
df = pd.read_csv(csv_file_path, dtype=dtype_dict)

# Data cleaning and type conversion
df = df.dropna(subset=["t", "x", "y"])
df["trackID"] = pd.to_numeric(df["trackID"], errors="coerce").astype("Int64")
df = df.dropna(subset=["trackID"])

# Data scaling
df["t"] *= s_per_frame
df["x"] *= um_per_pixel
df["y"] *= um_per_pixel

df[["trackID", "x", "y", "t"]]

,trackID,x,y,t
0,0,31.536149,15.366829,0.0
1,0,31.558399,15.383016,2.0
2,0,31.546452,15.359092,4.0
3,0,31.540225,15.358206,6.0
4,0,31.528332,15.361091,8.0
...,...,...,...,...
102864,1713,19.414646,22.615961,390.0
102865,1713,19.417402,22.583861,392.0
102866,1713,19.401713,22.588933,394.0
102867,1713,19.415493,22.580736,396.0


In [5]:
# groupby and sort operation:

grouped_tracks = df.groupby("trackID", group_keys=True).apply(lambda x: x.sort_values("t"))

for track_id, track_data in grouped_tracks.groupby(level=0):
    print(f"trackID: {track_id}")
    print(track_data)
    break

trackID: 0
                spotID  trackID     QUALITY          x          y      t    R  \
trackID                                                                         
0       0    2511848.0        0  552.495544  31.536149  15.366829    0.0  2.0   
        1    2515986.0        0  642.849365  31.558399  15.383016    2.0  2.0   
        2    2527827.0        0  642.203125  31.546452  15.359092    4.0  2.0   
        3    2507356.0        0  681.505310  31.540225  15.358206    6.0  2.0   
        4    2530496.0        0  658.882935  31.528332  15.361091    8.0  2.0   
...                ...      ...         ...        ...        ...    ...  ...   
        195  2793437.0        0  650.812256  31.211342  16.002479  390.0  2.0   
        196  2791335.0        0  642.328918  31.180615  15.994541  392.0  2.0   
        197  2794836.0        0  703.211304  31.180031  15.989657  394.0  2.0   
        198  2797707.0        0  650.146362  31.187152  15.993132  396.0  2.0   
        199  2796

In [6]:
def calc_MSD_NonPhysUnit(track_data, lags):

    Xs = track_data["x"].to_numpy()
    Ys = track_data["y"].to_numpy()

    MSDs = []
    for lag in lags:
        displacements = (Xs[:-lag] - Xs[lag:]) ** 2 + (Ys[:-lag] - Ys[lag:]) ** 2
        valid_displacements = displacements[~np.isnan(displacements)]
        MSD = np.nanmean(valid_displacements)
        MSDs.append(MSD)

    return np.array(MSDs, dtype=float)

In [7]:
for track_id, track_data in grouped_tracks.groupby(level=0):

    lags = np.arange(1, (window_size + 1))
    MSDs = calc_MSD_NonPhysUnit(track_data, lags)

    print(f"trackID : {track_id}")
    print(f"MSD: {MSDs}\n")

    break

trackID : 0
MSD: [0.00123688 0.00175779 0.00211803 0.00255088 0.00310741]



In [8]:
def calc_alpha(MSDs, lags):

    valid_indices = ~np.isnan(MSDs)
    valid_MSDs = MSDs[valid_indices]
    valid_lags = lags[valid_indices]

    log_lags = np.log10(valid_lags)
    log_MSDs = np.log10(valid_MSDs)

    slope, intercept, r_value, p_value, std_error = stats.linregress(log_lags, log_MSDs)

    alpha = slope
    diffusion_coefficient = (1 / 4) * (10**intercept)

    return alpha, r_value**2, diffusion_coefficient

In [9]:
for track_id, track_data in grouped_tracks.groupby(level=0):
    lags = np.arange(1, (window_size + 1))
    MSDs = calc_MSD_NonPhysUnit(track_data, lags)

    if not np.any(np.isnan(MSDs)):
        alpha, r_squared, diffusion_coefficient = calc_alpha(MSDs, lags)

        print(f"trackID: {track_id}")
        print(f"Alpha: {alpha}")
        print(f"R^2: {r_squared}")
        print(f"Diffusion Coefficient: {diffusion_coefficient}\n")

    break

trackID: 0
Alpha: 0.553357344419484
R^2: 0.9878037443853225
Diffusion Coefficient: 0.0003022014470825586



In [10]:
def calculate_alpha_and_D_for_track(df_track, um_per_pixel, s_per_frame, window_size):

    df_track["R2"] = np.nan
    df_track["alpha"] = np.nan
    df_track["D"] = np.nan

    window_info = []

    # Iterate through windows and update alpha for the middle frame
    step_size = 1
    for start in range(0, len(df_track) - window_size + 1, step_size):
        end = start + window_size
        df_window = df_track.iloc[start:end]

        number_lag = ceil(window_size / 2)
        if number_lag < 3:
            number_lag = 3
        window_msd = calc_MSD_NonPhysUnit(
            df_track.iloc[start:end], np.arange(1, number_lag + 1)
        )
        if np.sum(window_msd <= 0) > 0:
            # Skip this window since it contains invalid MSD values
            continue

        alpha, r_value, D = calc_alpha(
            window_msd, np.arange(1, number_lag + 1) * s_per_frame
        )
        r_squared = r_value**2
        if not np.isnan(alpha):
            middle_frame_index = start + ceil(window_size / 2)
            df_track.at[df_track.index[middle_frame_index], "R2"] = r_squared
            df_track.at[df_track.index[middle_frame_index], "alpha"] = alpha
            df_track.at[df_track.index[middle_frame_index], "D"] = D

            window_info.append(
                {
                    "window_start_frame": df_window.iloc[0]["t"],
                    "window_end_frame": df_window.iloc[-1]["t"],
                    "alpha": alpha,
                    "R2": r_squared,
                    "D": D,
                    "lags": np.arange(1, number_lag + 1).tolist(),
                }
            )

    return df_track, window_info

In [11]:
for track_id, df_track in grouped_tracks.groupby(level=0):

    result_df_track, window_info = calculate_alpha_and_D_for_track(
        df_track, um_per_pixel, s_per_frame, window_size
    )
    calculated = result_df_track.dropna(subset=["alpha", "R2", "D"])

    print(f"trackID: {track_id}")

    if not calculated.empty:
        print(
            calculated[["x", "y", "t", "R2", "alpha", "D"]]
        )
        for info in window_info:
            print(
                f"Window from {info['window_start_frame']} to {info['window_end_frame']}s: lags = {info['lags']}"
            )
            print(f"Alpha: {info['alpha']}, R2: {info['R2']}, D: {info['D']}\n")

    print("\n")

    break

trackID: 0
                     x          y      t        R2     alpha         D
trackID                                                               
0       3    31.540225  15.358206    6.0  0.690260  0.489589  0.000070
        4    31.528332  15.361091    8.0  0.985336  1.096968  0.000029
        5    31.532240  15.371324   10.0  0.778693  0.819952  0.000046
        6    31.558949  15.384844   12.0  0.989910  1.424520  0.000034
        7    31.555442  15.400241   14.0  0.984070  1.136391  0.000056
...                ...        ...    ...       ...       ...       ...
        194  31.200321  15.973766  388.0  0.002664 -0.041964  0.000284
        195  31.211342  16.002479  390.0  0.999709  0.194190  0.000233
        196  31.180615  15.994541  392.0  0.595969  0.602418  0.000089
        197  31.180031  15.989657  394.0  0.743288  0.261990  0.000109
        198  31.187152  15.993132  396.0  0.859873  0.299127  0.000130

[196 rows x 6 columns]
Window from 0.0 to 8.0s: lags = [1, 2, 3]


In [12]:
def process_csv_and_add_alpha_and_D(
    csv_file_path, window_size, um_per_pixel, s_per_frame
):

    df = pd.read_csv(csv_file_path, dtype=dtype_dict)
    df["t"] *= s_per_frame
    df["x"] *= um_per_pixel
    df["y"] *= um_per_pixel

    processed_track_list = []

    grouped_tracks = df.groupby("trackID")
    for track_id, df_track in grouped_tracks:
        processed_track, window_info = calculate_alpha_and_D_for_track(
            df_track, um_per_pixel, s_per_frame, window_size
        )
        processed_track_list.append(
            processed_track[
                [
                    "trackID",
                    "x",
                    "y",
                    "t",
                    "R2",
                    "alpha",
                    "D",
                ]
            ]
        )

    processed_df = pd.concat(processed_track_list).reset_index(drop=True)

    save_path = os.path.dirname(csv_file_path)
    base_name = os.path.basename(csv_file_path)
    name, ext = os.path.splitext(base_name)

    output_file_name = f"{name}_processed_alpha_and_D_w{window_size}{ext}"
    output_file_path = os.path.join(save_path, output_file_name)

    processed_df.to_csv(output_file_path, index=False)
    print(f"Processed CSV file saved: {output_file_path}")

    return output_file_path


process_csv_and_add_alpha_and_D(csv_file_path, window_size, um_per_pixel, s_per_frame)

Processed CSV file saved: Z:/Bisal_Halder_turbo/PROCESSED_DATA/Impact_of_cytoskeleton_on_HOPS_condensates/Nocodazole/20uM/Analysed Data/60min\20240319_UGD-2x-2s-20uM-Nocodazole-60min-replicate2-FOV-2_reformatted_processed_alpha_and_D_w5.csv


'Z:/Bisal_Halder_turbo/PROCESSED_DATA/Impact_of_cytoskeleton_on_HOPS_condensates/Nocodazole/20uM/Analysed Data/60min\\20240319_UGD-2x-2s-20uM-Nocodazole-60min-replicate2-FOV-2_reformatted_processed_alpha_and_D_w5.csv'